In [10]:
import torch
from collections import OrderedDict
from basicsr.models import create_model
from basicsr.utils.options import parse
import numpy as np
from tqdm import tqdm
from basicsr.data import create_dataloader, create_dataset
from tqdm.notebook import tqdm as log_progress
from torch.utils.data import TensorDataset, DataLoader

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device('cuda:0')

In [4]:
lambdas = torch.linspace(0,10,500)


In [5]:
#loss could be MSE
def fraction_missed_loss(pset,label):
    misses = (pset[0].squeeze() > label[:, 1:2, :, :].squeeze()).float() + (pset[2].squeeze() < label[:,1:2, :, :].squeeze()).float()
    misses[misses > 1.0] = 1.0
    d = len(misses.shape)
#     print('misses shape = ', misses.shape)
#     print('d = ', d)
#     print('np sum missed = ', torch.sum(misses))
#     print(len(pset))
#     print(len(label))
#     print("returning = ", misses.mean(dim=tuple(range(1,d))))
    return misses.mean(dim=tuple(range(1,d)))

def get_rcps_loss_fn(string):
    if string == 'fraction_missed':

        #return torch.nn.MSELoss()
        return fraction_missed_loss
    else:
        raise NotImplementedError

rcps_loss_fn = get_rcps_loss_fn('fraction_missed')



In [6]:
def inn_nested_sets_from_output(model, output, lam=None):
    output[:,0,:,:] = torch.minimum(output[:,0,:,:], output[:,1,:,:]-1e-6)
    output[:,2,:,:] = torch.maximum(output[:,2,:,:], output[:,1,:,:]+1e-6)
    
    upper_edge = lam * (output[:,2,:,:] - output[:,1,:,:]) + output[:,1,:,:] 
    lower_edge = output[:,1,:,:] - lam * (output[:,1,:,:] - output[:,0,:,:])
    return lower_edge, output[:,1,:,:], upper_edge 

# Always outputs [0,1] valued nested sets
def nested_sets_from_output(output, lam=None):
    lower_edge, prediction, upper_edge = inn_nested_sets_from_output(model, output, lam)
    
#     upper_edge = torch.maximum(upper_edge, prediction + 1e-6) # set a lower bound on the size.
#     lower_edge = torch.minimum(lower_edge, prediction - 1e-6)
    
    return lower_edge, prediction, upper_edge 

def get_rcps_losses_from_outputs(model, out_dataset, rcps_loss_fn, lam, device):
    losses = []
    dataloader = DataLoader(out_dataset, batch_size=64, shuffle=False, num_workers=0, pin_memory=False) 
    model = model.to(device)
    for batch in dataloader:
        #print('x shape', x.shape)
        x, labels = batch
        sets = nested_sets_from_output(x,lam) 
        #print('sets shape:', sets[0].shape)
        #print('labels shape:', labels.to(device).cpu().shape)
        losses = losses + [rcps_loss_fn(sets, labels.to(device)).cpu(),]
        #print('losses = ', losses)
    return torch.cat(losses,dim=0)



In [7]:
def get_lhat(calib_loss_table, lambdas, alpha, B=1):
    #calib_loss_table = calib_loss_table
    n = calib_loss_table.shape[0]
    #print(n)
    rhat = calib_loss_table.mean(axis=0)
#     print(len(rhat))
#     print(rhat)
    rhat = torch.flip(rhat, dims = (0,))
    lambdas = torch.flip(lambdas, dims = (0,))
#     print('rhat = ', rhat)
#     print(((n/(n+1)) * rhat  + B/(n+1)) >= alpha)
    lhat_idx = max(np.argmax(((n/(n+1)) * rhat  + B/(n+1)) >= alpha) - 1, 0) # Can't be -1.
#     print('lhat_idx = ', lhat_idx)
    return lambdas[lhat_idx]

In [11]:
#SIDD: NAFNet-SIDD-08-28-NEW     76k   //USE 50k instead
#FMD: NAFNet-FMD-8-28

#SIDD/SHG: SIDD-SHG-08-29        27k
#FMD/SHG: FMD-SHG-08-29

finetuned_weight_path = '/home/gridsan/tye/NAFNet/experiments/NAFNet-FMD-20-stack-NORMALIZED/models/net_g_125000.pth'
finetuned_model_weights = torch.load(finetuned_weight_path)


opt_path = '/home/gridsan/tye/NAFNet/options/train/SIDD/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False

finetuned = create_model(opt) #Finetuned on SIDD
 

finetuned = finetuned.net_g


finetuned.load_state_dict(finetuned_model_weights['params'])
print('finetuned weights loaded successfully')

model = finetuned.to(device)



 opt[network_g] =  OrderedDict([('type', 'NAFNet'), ('width', 64), ('enc_blk_nums', [2, 2, 4, 8]), ('middle_blk_num', 12), ('dec_blk_nums', [2, 2, 2, 2])])

img_channel =  20 



2024-04-30 20:14:38,494 INFO: Model [ImageRestorationModel] is created.


finetuned weights loaded successfully


In [12]:
# i = 19
# name = "20_stacked_combo_calib_" + str(i)

# OPT =  OrderedDict([('name', 'NAFNet-SIDD-SHG-08-21-TEST'), ('model_type', 'ImageRestorationModel'), ('scale', 1), ('num_gpu', 1), ('manual_seed', 10), ('datasets', OrderedDict([('train', OrderedDict([('name', 'COMBO'), ('type', 'PairedImageDataset'), ('dataroot_gt', './datasets/' + name +'/train/gt_crops.lmdb'), ('dataroot_lq', './datasets/' + name + '/train/input_crops.lmdb'), ('filename_tmpl', '{}'), ('io_backend', OrderedDict([('type', 'lmdb')])), ('gt_size', 256), ('use_flip', False), ('use_rot', False), ('use_shuffle', True), ('num_worker_per_gpu', 1), ('batch_size_per_gpu', 1), ('dataset_enlarge_ratio', 1), ('prefetch_mode', None), ('phase', 'train'), ('scale', 1)])), ('val', OrderedDict([('name', 'COMBO_val'), ('type', 'PairedImageDataset'), ('dataroot_gt', './datasets/' + name + '/val/gt_crops.lmdb'), ('dataroot_lq', './datasets/' + name + '/val/input_crops.lmdb'), ('io_backend', OrderedDict([('type', 'lmdb')])), ('phase', 'val'), ('scale', 1)]))])), ('network_g', OrderedDict([('type', 'NAFNet'), ('width', 64), ('enc_blk_nums', [2, 2, 4, 8]), ('middle_blk_num', 12), ('dec_blk_nums', [2, 2, 2, 2])])), ('path', OrderedDict([('pretrain_network_g', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-08-19-Different-5-Channel/models/net_g_100000.pth'), ('strict_load_g', True), ('resume_state', None), ('root', '/home/gridsan/tye/NAFNet'), ('experiments_root', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-SHG-08-21-TEST'), ('models', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-SHG-08-21-TEST/models'), ('training_states', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-SHG-08-21-TEST/training_states'), ('log', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-SHG-08-21-TEST'), ('visualization', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-SHG-08-21-TEST/visualization')])), ('train', OrderedDict([('optim_g', OrderedDict([('type', 'AdamW'), ('lr', 0.001), ('weight_decay', 0.0), ('betas', [0.9, 0.9])])), ('scheduler', OrderedDict([('type', 'TrueCosineAnnealingLR'), ('T_max', 400000), ('eta_min', 1e-07)])), ('total_iter', 400000), ('warmup_iter', -1), ('pixel_opt', OrderedDict([('type', 'PSNRLoss'), ('loss_weight', 1), ('reduction', 'mean')]))])), ('val', OrderedDict([('val_freq', 20000.0), ('save_img', False), ('use_image', False), ('metrics', OrderedDict([('psnr', OrderedDict([('type', 'calculate_psnr'), ('crop_border', 0), ('test_y_channel', False)])), ('ssim', OrderedDict([('type', 'calculate_ssim'), ('crop_border', 0), ('test_y_channel', False)]))]))])), ('logger', OrderedDict([('print_freq', 200), ('save_checkpoint_freq', 1000.0), ('use_tb_logger', True), ('wandb', OrderedDict([('project', None), ('resume_id', None)]))])), ('dist_params', OrderedDict([('backend', 'nccl'), ('port', 29500)])), ('is_train', True), ('dist', True), ('rank', 0), ('world_size', 1)])

# for phase, dataset_opt in OPT['datasets'].items():
#         if phase == 'train':
#             dataset_enlarge_ratio = dataset_opt.get('dataset_enlarge_ratio', 1)
#             dataset = create_dataset(dataset_opt)

In [13]:
# print(dataset[0]['gt'].shape)

# labels_shape = list(torch.tensor(dataset[0]['gt']).unsqueeze(0).shape)

# labels_shape[0] = len(dataset)

# labels = torch.zeros(tuple(labels_shape)) #, device =device)

# outputs_shape = list(model(torch.tensor(dataset[0]['lq']).unsqueeze(0).to(device)).shape)

# outputs_shape[0] = len(dataset)

# outputs = torch.zeros(tuple(outputs_shape), device =device)

# tempDL = DataLoader(dataset, num_workers=0, batch_size=1, pin_memory=True)

# counter = 0
# for batch in tqdm(tempDL):
#      with torch.no_grad():
#             #print('batch = ',  batch['lq'].shape[0])
#             outputs[counter:counter+batch['lq'].shape[0],:,:,:] = model(batch['lq'].to(device))
#             labels[counter:counter+batch['gt'].shape[0]] = batch['gt']
#             counter += batch['lq'].shape[0]
#             #if counter == 100:
#             #    break

In [11]:
print("Output dataset")
labels_test = torch.mean(labels, 1).unsqueeze(1)

labels_new = labels_test.repeat((1, 3, 1, 1))

out_dataset = TensorDataset(outputs,labels_new.cpu())
dlambda = lambdas[1]-lambdas[0]
#model.set_lhat(lambdas[-1]+dlambda-1e-9)
print("Computing losses")
calib_loss_table = torch.zeros((outputs.shape[0],lambdas.shape[0]))
for lam in log_progress(lambdas):
    #print(rcps_loss_fn.shape)
    losses = get_rcps_losses_from_outputs(model, out_dataset, rcps_loss_fn, lam-dlambda, device)
    #print()
    calib_loss_table[:,np.where(lambdas==lam)[0]] = losses[:,None]
    

Output dataset
Computing losses


In [12]:
print("lhat = ", get_lhat(calib_loss_table, lambdas, alpha = 0.1).numpy())

lhat =  3.3066132


In [21]:
lhats = {}

channel = "calib_0422_ch4"
for i in range(1, 21):
    num = str(i).zfill(2)
    OPT =  OrderedDict([('name', 'NAFNet-SIDD-SHG-08-21-TEST'), ('model_type', 'ImageRestorationModel'), ('scale', 1), ('num_gpu', 1), ('manual_seed', 10), ('datasets', OrderedDict([('train', OrderedDict([('name', 'COMBO'), ('type', 'PairedImageDataset'), ('dataroot_gt', './datasets/' + channel + "/" + num +'/train/gt_crops.lmdb'), ('dataroot_lq', './datasets/' + channel + "/" + num +'/train/input_crops.lmdb'), ('filename_tmpl', '{}'), ('io_backend', OrderedDict([('type', 'lmdb')])), ('gt_size', 256), ('use_flip', False), ('use_rot', False), ('use_shuffle', True), ('num_worker_per_gpu', 1), ('batch_size_per_gpu', 1), ('dataset_enlarge_ratio', 1), ('prefetch_mode', None), ('phase', 'train'), ('scale', 1)])), ('val', OrderedDict([('name', 'COMBO_val'), ('type', 'PairedImageDataset'), ('dataroot_gt', './datasets/' + channel + "/" + num +'/val/gt_crops.lmdb'), ('dataroot_lq', './datasets/' + name + '/val/input_crops.lmdb'), ('io_backend', OrderedDict([('type', 'lmdb')])), ('phase', 'val'), ('scale', 1)]))])), ('network_g', OrderedDict([('type', 'NAFNet'), ('width', 64), ('enc_blk_nums', [2, 2, 4, 8]), ('middle_blk_num', 12), ('dec_blk_nums', [2, 2, 2, 2])])), ('path', OrderedDict([('pretrain_network_g', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-08-19-Different-5-Channel/models/net_g_100000.pth'), ('strict_load_g', True), ('resume_state', None), ('root', '/home/gridsan/tye/NAFNet'), ('experiments_root', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-SHG-08-21-TEST'), ('models', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-SHG-08-21-TEST/models'), ('training_states', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-SHG-08-21-TEST/training_states'), ('log', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-SHG-08-21-TEST'), ('visualization', '/home/gridsan/tye/NAFNet/experiments/NAFNet-SIDD-SHG-08-21-TEST/visualization')])), ('train', OrderedDict([('optim_g', OrderedDict([('type', 'AdamW'), ('lr', 0.001), ('weight_decay', 0.0), ('betas', [0.9, 0.9])])), ('scheduler', OrderedDict([('type', 'TrueCosineAnnealingLR'), ('T_max', 400000), ('eta_min', 1e-07)])), ('total_iter', 400000), ('warmup_iter', -1), ('pixel_opt', OrderedDict([('type', 'PSNRLoss'), ('loss_weight', 1), ('reduction', 'mean')]))])), ('val', OrderedDict([('val_freq', 20000.0), ('save_img', False), ('use_image', False), ('metrics', OrderedDict([('psnr', OrderedDict([('type', 'calculate_psnr'), ('crop_border', 0), ('test_y_channel', False)])), ('ssim', OrderedDict([('type', 'calculate_ssim'), ('crop_border', 0), ('test_y_channel', False)]))]))])), ('logger', OrderedDict([('print_freq', 200), ('save_checkpoint_freq', 1000.0), ('use_tb_logger', True), ('wandb', OrderedDict([('project', None), ('resume_id', None)]))])), ('dist_params', OrderedDict([('backend', 'nccl'), ('port', 29500)])), ('is_train', True), ('dist', True), ('rank', 0), ('world_size', 1)])

    for phase, dataset_opt in OPT['datasets'].items():
            if phase == 'train':
                dataset_enlarge_ratio = dataset_opt.get('dataset_enlarge_ratio', 1)
                dataset = create_dataset(dataset_opt)
                
    labels_shape = list(torch.tensor(dataset[0]['gt']).unsqueeze(0).shape)

    labels_shape[0] = len(dataset)

    labels = torch.zeros(tuple(labels_shape)) #, device =device)

    outputs_shape = list(model(torch.tensor(dataset[0]['lq']).unsqueeze(0).to(device)).shape)

    outputs_shape[0] = len(dataset)

    outputs = torch.zeros(tuple(outputs_shape), device =device)

    tempDL = DataLoader(dataset, num_workers=0, batch_size=1, pin_memory=True)

    counter = 0
    for batch in tqdm(tempDL):
         with torch.no_grad():
                #print('batch = ',  batch['lq'].shape[0])
                outputs[counter:counter+batch['lq'].shape[0],:,:,:] = model(batch['lq'].to(device))
                labels[counter:counter+batch['gt'].shape[0]] = batch['gt']
                counter += batch['lq'].shape[0]
                
    labels_test = torch.mean(labels, 1).unsqueeze(1)

    labels_new = labels_test.repeat((1, 3, 1, 1))

    out_dataset = TensorDataset(outputs,labels_new.cpu())
    dlambda = lambdas[1]-lambdas[0]
    #model.set_lhat(lambdas[-1]+dlambda-1e-9)
    print("Computing losses")
    calib_loss_table = torch.zeros((outputs.shape[0],lambdas.shape[0]))
    for lam in log_progress(lambdas):
        #print(rcps_loss_fn.shape)
        losses = get_rcps_losses_from_outputs(model, out_dataset, rcps_loss_fn, lam-dlambda, device)
        #print()
        calib_loss_table[:,np.where(lambdas==lam)[0]] = losses[:,None]
        
    lhats[i] = get_lhat(calib_loss_table, lambdas, alpha = 0.1).numpy()
        

2024-04-30 20:52:22,694 INFO: Dataset PairedImageDataset - COMBO is created.


\input key =  lq
\gt key =  gt

INPUT LMDB KEYS =  ['/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/input_crops/33_01_s001.pt', '/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/input_crops/34_01_s001.pt', '/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/input_crops/35_01_s001.pt', '/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/input_crops/36_01_s001.pt', '/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/input_crops/37_01_s001.pt', '/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/input_crops/38_01_s001.pt', '/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/input_crops/39_01_s001.pt', '/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/input_crops/40_01_s001.pt', '/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/input_crops/41_01_s001.pt', '/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/input_crops/42_01_s001.pt']

GT LMDB KEYS =  ['/home/gridsan/tye/NAFNet/datasets/calib_0422_ch4/01/train/g

  0%|          | 0/31 [00:00<?, ?it/s]

LQ IMAGE SHAPE:  (20, 500, 500)
LQ IMAGE SHAPE:  (20, 500, 500)
LQ IMAGE SHAPE:  (20, 500, 500)


  6%|▋         | 2/31 [00:02<00:29,  1.02s/it]

...

LQ IMAGE SHAPE:  (20, 500, 500)


 97%|█████████▋| 31/32 [00:22<00:00,  1.12it/s]

LQ IMAGE SHAPE:  (20, 500, 500)


100%|██████████| 32/32 [00:24<00:00,  1.33it/s]

LQ IMAGE SHAPE:  (20, 500, 500)
Computing losses


In [22]:
lhats_list = [value.item() for value in lhats.values()]
print(lhats_list)

[2.525049924850464, 2.7254509925842285, 2.505009889602661, 2.685370683670044, 2.9058115482330322, 3.1462924480438232, 2.845691442489624, 2.685370683670044, 2.8056113719940186, 2.765531063079834, 3.2064127922058105, 3.086172103881836, 2.9058115482330322, 2.765531063079834, 2.7454910278320312, 2.7454910278320312, 2.4649298191070557, 2.424849510192871, 2.0841684341430664, 1.9639278650283813]


In [ ]:

ch1_lambdas = [6.212425231933594, 7.675350666046143, 8.296592712402344, 8.97795581817627, 8.176352500915527, 8.897795677185059, 6.953907489776611, 7.314629077911377, 8.797595024108887, 10.0, 10.0, 8.396794319152832, 7.094188213348389, 6.673346519470215, 7.575150012969971, 7.795590877532959, 9.859719276428223, 8.476954460144043, 5.8116230964660645, 8.897795677185059]

ch2_lambdas =[10.0, 10.0, 7.995992183685303, 6.352705001831055, 5.1503005027771, 4.589178085327148, 4.448897838592529, 4.52905797958374, 4.629258632659912, 3.567134141921997, 3.3867735862731934, 3.647294521331787, 3.3667335510253906, 3.1863725185394287, 4.008016109466553, 4.609218597412109, 3.887775421142578, 5.751502990722656, 4.729458808898926, 4.889779090881348]

ch3_lambdas = [1.302605152130127, 1.2825651168823242, 1.302605152130127, 1.2224448919296265, 1.1623246669769287, 1.0821642875671387, 1.0420842170715332, 0.9418837428092957, 0.8617234230041504, 0.8216432929039001, 0.7414829730987549, 0.7214428782463074, 0.7414829730987549, 0.7014027833938599, 0.6813627481460571, 0.6813627481460571, 0.7014027833938599, 0.6613226532936096, 0.6212424635887146, 0.6012023687362671]

ch4_lambdas = [2.525049924850464, 2.7254509925842285, 2.505009889602661, 2.685370683670044, 2.9058115482330322, 3.1462924480438232, 2.845691442489624, 2.685370683670044, 2.8056113719940186, 2.765531063079834, 3.2064127922058105, 3.086172103881836, 2.9058115482330322, 2.765531063079834, 2.7454910278320312, 2.7454910278320312, 2.4649298191070557, 2.424849510192871, 2.0841684341430664, 1.9639278650283813]


In [15]:
#ONE
SIDD = 1.8036072254180908    #1.8637274503707886
FMD= 2.024048089981079
SIDD_SHG = 2.5651302337646484
FMD_SHG= 1.6432865858078003

In [16]:
#THREE
SIDD = 1.7835670709609985
FMD= 3.1462924480438232
SIDD_SHG = 2.4649298191070557
FMD_SHG= 1.8036072254180908

In [17]:
#FIVE
SIDD = 3.1863725185394287
FMD= 4.088176250457764
SIDD_SHG = 2.3847694396972656
FMD_SHG= 1.7234468460083008

In [18]:
#THIS WAS WITH THE COMBINED IMAGES
# SIDD = 2.525049924850464
# FMD= 5.591182708740234
# SIDD_SHG = 2.2244489192962646
# FMD_SHG= 1.6833667755126953